# Train Sequential Neural Network

In [2]:
import re
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Lambda, Dropout

import os
import numpy as np


# load in text 
raw_text = open(os.path.join(os.getcwd(), 'data_shakespeare/shakespeare.txt')).read()
lines = [line.split() for line in raw_text.split('\n\n') if line.split()]

# remove all unnecessary characters from the text
raw_text2 = ''
for line in lines:
    obs_elem = []
    for word in line:
        word = re.sub("\d+", "", word)
        if (word == ""):
            continue
        word = re.sub(r'[^-\w\']', '', word).lower()
        raw_text2 += word + ' '
            
# create same-length strings 
length = 40
# get list of all characters used in text
chars = sorted(list(set(raw_text2)))
# map characters to their numerical value
mapping = dict((c, i) for i, c in enumerate(chars))
vocab_size = len(mapping)
print(len(raw_text2))
# tokenize a 40 length sequence and the character
# coming after it (the 40 length sequence will be 
# x and the 41st character will be y)
X = []
y = []
# using semi-redundant sequences to speed up training
for i in range(length, len(raw_text2) - 1):
    # select sequence of tokens
    seq = raw_text2[i-length:i+1]
    # store
    encoded_seq = [mapping[c] for c in list(seq)]
    X.append(np.array(encoded_seq))
    output = raw_text2[i + 1]
    encoded_seq2 = mapping[output]
    y.append(encoded_seq2)
    
# separate into input and output
X = np.array(X)
y = np.array(y)
beforepX = X
# converts x and y to binary class matrices (depending on 
# if the character is present in that sequence)
newsequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(newsequences)
y = to_categorical(y, num_classes=vocab_size)



Using TensorFlow backend.


91006


# Generate poem from Sequential Neural Network

In [3]:
# source that I used to help generate this code:
# https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/


def generate_poem(model, chars):
    ''' passing in a trained model and all the characters
    that the model was trained on, generates a poem based
    on model predictions '''
    # get random initial line
    start = np.random.randint(0, len(beforepX)-1)
    # dictionary converting integers to characters
    # (since we will generate predictions that are int sequences,
    # we convert them to characters)
    int_to_char = dict((i, c) for i, c in enumerate(chars))
    # get a random seed from the input data
    pattern = list(beforepX[start])
    # generate characters
    # based on the input data, generate predictions 
    poem = ''
    for k in range(700):
        # get a prediction based on the pattern for what 
        # the next character will be 
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = to_categorical(x, num_classes=vocab_size)
        prediction = model.predict(x, verbose=0)
        
        # get prediction with highest probability 
        index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        poem = poem + result
        
        # append that predicted pattern, remove first character 
        # from predicted pattern
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    # randomly add punctuation to poem, print poem out
    punctuation_list = [',', '.', ':', '?']
    punctuation_probs = [0.6, 0.1, 0.2, 0.1]
    poem_format = poem.split()
    for p in range(1, len(poem_format)):
        if ((p - 1) % 10 == 0):
            print(poem_format[p].capitalize() + ' ', end = '')
        else:
            if (poem_format[p] == 'i'):
                print(poem_format[p].capitalize() + ' ', end = '')
            else:
                print(poem_format[p], end = '')
            if (p % 10 == 0 and p < len(poem_format) - 1):
                print(np.random.choice(punctuation_list, p = punctuation_probs) + ' ', end = '')
                print()
            elif(p == len(poem_format) - 1):
                print('.', end = '')
            else:
                print(' ', end = '')
        
            
def summers_day_poem(model, chars):
    ''' passing in a trained model and all the characters
    that the model was trained on, generates a poem based
    on model predictions -- the input is set to be shall
    i compare thee to a summers day'''
        
    test = "shall i compare thee to a summers day tho"
    test = re.sub("\d+", " ", test)
    test = re.sub(r'[^-\w\']', ' ', test).lower()
    pattern = [mapping[c] for c in list(test)]
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    # generate characters
    poem = ''
    for k in range(700):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = to_categorical(x, num_classes=vocab_size)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        poem = poem + result
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    punctuation_list = [',', '.', ':', '?']
    punctuation_probs = [0.6, 0.1, 0.2, 0.1]
    poem_format = poem.split()
    for p in range(1, len(poem_format)):
        if ((p - 1) % 10 == 0):
            print(poem_format[p].capitalize() + ' ', end = '')
        else:
            if (poem_format[p] == 'i'):
                print(poem_format[p].capitalize() + ' ', end = '')
            else:
                print(poem_format[p], end = '')
            if (p % 10 == 0 and p < len(poem_format) - 1):
                print(np.random.choice(punctuation_list, p = punctuation_probs) + ' ', end = '')
                print()
            elif(p == len(poem_format) - 1):
                print('.', end = '')
            else:
                print(' ', end = '')

# Temperature = 1.5

In [7]:
model1 = Sequential()
model1.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model1.add(Dropout(0.3))
model1.add(Dense(200, input_shape=(X.shape[1], X.shape[2])))
model1.add(Dropout(0.3))
model1.add(Dense(200, input_shape=(X.shape[1], X.shape[2])))
model1.add(Dropout(0.3))

# adding temperature
temp = 1.5
model1.add(Lambda(lambda x : x /temp))
model1.add(Dense(vocab_size, activation='softmax'))

print(model1.summary())
# compile model
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

model1.fit(X, y, epochs=200, verbose=2)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 200)               184000    
_________________________________________________________________
dropout_7 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_8 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_9 (Dropout)          (None, 200)               0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 200)               0         
__________

Epoch 137/200
 - 236s - loss: 0.7837 - acc: 0.7475
Epoch 138/200
 - 258s - loss: 0.7790 - acc: 0.7483
Epoch 139/200
 - 262s - loss: 0.7783 - acc: 0.7490
Epoch 140/200
 - 257s - loss: 0.7755 - acc: 0.7490
Epoch 141/200
 - 280s - loss: 0.7767 - acc: 0.7480
Epoch 142/200
 - 218s - loss: 0.7772 - acc: 0.7479
Epoch 143/200
 - 237s - loss: 0.7716 - acc: 0.7512
Epoch 144/200
 - 224s - loss: 0.7689 - acc: 0.7502
Epoch 145/200
 - 203s - loss: 0.7661 - acc: 0.7518
Epoch 146/200
 - 204s - loss: 0.7686 - acc: 0.7517
Epoch 147/200
 - 203s - loss: 0.7639 - acc: 0.7526
Epoch 148/200
 - 204s - loss: 0.7619 - acc: 0.7520
Epoch 149/200
 - 206s - loss: 0.7599 - acc: 0.7531
Epoch 150/200
 - 205s - loss: 0.7621 - acc: 0.7524
Epoch 151/200
 - 205s - loss: 0.7581 - acc: 0.7546
Epoch 152/200
 - 204s - loss: 0.7606 - acc: 0.7530
Epoch 153/200
 - 206s - loss: 0.7600 - acc: 0.7547
Epoch 154/200
 - 206s - loss: 0.7569 - acc: 0.7554
Epoch 155/200
 - 205s - loss: 0.7556 - acc: 0.7567
Epoch 156/200
 - 204s - loss: 0

In [8]:
generate_poem(model1, chars)

If are another's green and the parted and the parts, 
To make of mine and still than thou art other, 
In their beauty of thee thine and so to time, 
Do his sporttous mine own self to truth of many: 
My heart which in my dear still and yet so. 
Should ach that thou hast stall that I  have starn, 
To most of love that deel'st thou to thee shall: 
Star for love though not so but decay of all, 
To the heans and write and master that thou art, 
I love thee all the world will be it leap: 
To every to make and my dear still and yet, 
So is the time that beauty still and you and: 
Thee in my dode in and my self a bereater, 
Brand shine and my self alone as my love to? 
Make me dead that thou shouldst not and the summer, 
Sweet 

In [9]:
summers_day_poem(model1, chars)

But the time and to my self a moning tor, 
Shall self alone and thou art the earth says so, 
Say more reason of your true love to show what: 
Wealth she can all their state but do not so, 
I love thee in must barly love and they in. 
Though not so if thou art forsuned in thy soul. 
Knows is as a false of heart is thine eye. 
But thought compare thou art as thom a face and, 
Swear thou be dost thou to make my self a: 
Moner of this beauty shall stall in love to stay, 
I loved o hermakes so things removed that fies ence, 
As I  love thee all then make the world should: 
Loved and so I  love thee and all my some: 
Soon thou dost thou those alone this s in thee: 
As thou wilt for the sun of this are though, 
I will o love.

# Temperature = 0.75

In [3]:
model2 = Sequential()
model2.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model2.add(Dropout(0.3))
model2.add(Dense(200, input_shape=(X.shape[1], X.shape[2])))
model2.add(Dropout(0.3))
model2.add(Dense(200, input_shape=(X.shape[1], X.shape[2])))
model2.add(Dropout(0.3))

# adding temperature
temp = 0.75
model2.add(Lambda(lambda x : x /temp))
model2.add(Dense(vocab_size, activation='softmax'))

print(model2.summary())
# compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

model2.fit(X, y, epochs=200, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               184000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 200)               0         
__________

Epoch 137/200
 - 219s - loss: 0.8044 - acc: 0.7416
Epoch 138/200
 - 218s - loss: 0.8047 - acc: 0.7424
Epoch 139/200
 - 219s - loss: 0.7954 - acc: 0.7459
Epoch 140/200
 - 217s - loss: 0.7973 - acc: 0.7454
Epoch 141/200
 - 220s - loss: 0.7917 - acc: 0.7454
Epoch 142/200
 - 223s - loss: 0.7952 - acc: 0.7441
Epoch 143/200
 - 219s - loss: 0.7869 - acc: 0.7479
Epoch 144/200
 - 219s - loss: 0.7933 - acc: 0.7446
Epoch 145/200
 - 219s - loss: 0.7939 - acc: 0.7451
Epoch 146/200
 - 219s - loss: 0.7853 - acc: 0.7495
Epoch 147/200
 - 221s - loss: 0.7881 - acc: 0.7478
Epoch 148/200
 - 219s - loss: 0.7805 - acc: 0.7493
Epoch 149/200
 - 218s - loss: 0.7845 - acc: 0.7480
Epoch 150/200
 - 219s - loss: 0.7841 - acc: 0.7474
Epoch 151/200
 - 218s - loss: 0.7793 - acc: 0.7503
Epoch 152/200
 - 221s - loss: 0.7790 - acc: 0.7501
Epoch 153/200
 - 217s - loss: 0.7768 - acc: 0.7496
Epoch 154/200
 - 218s - loss: 0.7722 - acc: 0.7522
Epoch 155/200
 - 219s - loss: 0.7756 - acc: 0.7512
Epoch 156/200
 - 218s - loss: 0

In [4]:
generate_poem(model2, chars)

Thou art all the world and summer's floend which should. 
That which it be astell on my love being my, 
Part and therefore not so I  love thee in the, 
Part and will be but you may in my mind, 
That from the call of thee this spring which nightly: 
Graces that writ in one thing and therefore to be. 
Thence and straight and or cold and true and lives, 
Thoughts and beauty lardenhig ward and beauty laid in heaven, 
Is so stang as thou shalt be old which her: 
Before shall beauteous seem in the wide with teems that, 
Find the least that I  am not love that love: 
Is beauty not before than gavelest seeming that thou art. 
Therefore to be as a stars and worth to see: 
To the stars and something breast and sometime and.

In [5]:
summers_day_poem(model2, chars)

Art thou art as thoughts and theast thou beauteous nigmanarale, 
May no make in then make more let me so, 
Not thou mayst to be as plack wherite hold in, 
Them in the mind of me to be so shall: 
I spend in memory the mine eye say to the? 
Terming strange that in the eyes ware still the praise, 
And sometime that first is black and lovest fairent whine, 
I am statious not before thences and love to make? 
Me love is are the mained face shall be but. 
From the say their hers very fine or heaven thou: 
Art and say it to my verse to his shade, 
When it is all alone to each to know my, 
Friend's hust before to be as as as the truant. 
In the pearly drat the baak straight in seeming the, 
Self are not my live them for my s.

# Temperature = 0.25

In [5]:
model3 = Sequential()
model3.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model3.add(Dropout(0.3))
model3.add(Dense(200, input_shape=(X.shape[1], X.shape[2])))
model3.add(Dropout(0.3))
model3.add(Dense(200, input_shape=(X.shape[1], X.shape[2])))
model3.add(Dropout(0.3))

# adding temperature
temp = 0.25
model3.add(Lambda(lambda x : x /temp))
model3.add(Dense(vocab_size, activation='softmax'))

print(model3.summary())
# compile model
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

model3.fit(X, y, epochs=200, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 200)               184000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 200)               0         
__________

Epoch 137/200
 - 209s - loss: 0.8456 - acc: 0.7305
Epoch 138/200
 - 210s - loss: 0.8421 - acc: 0.7315
Epoch 139/200
 - 213s - loss: 0.8425 - acc: 0.7323
Epoch 140/200
 - 221s - loss: 0.8420 - acc: 0.7320
Epoch 141/200
 - 221s - loss: 0.8392 - acc: 0.7312
Epoch 142/200
 - 221s - loss: 0.8397 - acc: 0.7336
Epoch 143/200
 - 220s - loss: 0.8309 - acc: 0.7343
Epoch 144/200
 - 219s - loss: 0.8328 - acc: 0.7357
Epoch 145/200
 - 220s - loss: 0.8312 - acc: 0.7341
Epoch 146/200
 - 236s - loss: 0.8269 - acc: 0.7361
Epoch 147/200
 - 220s - loss: 0.8291 - acc: 0.7367
Epoch 148/200
 - 216s - loss: 0.8235 - acc: 0.7367
Epoch 149/200
 - 218s - loss: 0.8276 - acc: 0.7365
Epoch 150/200
 - 216s - loss: 0.8243 - acc: 0.7379
Epoch 151/200
 - 239s - loss: 0.8218 - acc: 0.7364
Epoch 152/200
 - 244s - loss: 0.8239 - acc: 0.7379
Epoch 153/200
 - 216s - loss: 0.8184 - acc: 0.7388
Epoch 154/200
 - 215s - loss: 0.8149 - acc: 0.7398
Epoch 155/200
 - 238s - loss: 0.8148 - acc: 0.7409
Epoch 156/200
 - 220s - loss: 0

In [6]:
generate_poem(model3, chars)

Thine own sweet self alone so shall think that time: 
With the canst not part and what is to let: 
Me fair lest thy part that I  this is the: 
Praise the earth a drought to be the grave though, 
All the less your sweet self alone should be as, 
I make shall best I  may I  sing when I , 
Say me then make hap you may I  make my, 
Love as tell and sake and I  may not remove? 
Nor me thine in their spile contents to time thou? 
Wert to this proud again as an and all thing: 
That thou shalt fair show that which it shall I : 
Swear means the praise then make the lease thet fair, 
To thou thy self desire thou with to make me, 
Ground the fairest in my love and the conceation dide, 
The parter me to my love may still shine when, 
I 

In [7]:
summers_day_poem(model3, chars)

Alters rook so like a glasted with the lines of, 
Love my dead to love withing all my body steep, 
To bear my tongue-tied my best thou wilt for they. 
Be old sight my love is their hard and so: 
Thou thy self desire thou wilt for I  may not: 
Proved and in the right for shame dear love thee, 
So store thee in the pride the painter with thy? 
Sweet sight and shadow shall be as thine thou art, 
As I  think the lears the strength on thy self: 
To bear what leave the counters an erpoctoor but for, 
The list and false more receives o chary the worght, 
That in the present doth that with thee the place. 
When I  should your self in my love's gain and, 
Praises of my love to come in the store to, 
Hide when in the spier with p.